# Part 02
------------------------------------------------------------





본 02 파트에서 데이터 전처리를 다음과 같이 수행했습니다.


# 📘 목차
1️⃣ [상품 기본 스펙 데이터](#-1️⃣-상품-기본-스펙-데이터) <br>
2️⃣ [품질 / 가격 데이터](#-2️⃣-품질--가격-지표) <br>
3️⃣ [생산 / 유통 메타 데이터](#-3️⃣-생산--유통-메타-데이터) <br>
4️⃣ [소비 경험 기록 데이터](#-4️⃣-소비-경험-기록-데이터) <br>

# 데이터 불러오기

In [17]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import gc

import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)
%matplotlib inline

!pip install koreanize-matplotlib
import koreanize_matplotlib

In [19]:
df = pd.read_csv('/content/drive/MyDrive/week_hands_on_project5/whiskey/dataset/whisky_data.csv')
df.columns = df.columns.str.lower()
df.head()

,name,main_info,volume_alcohol,price,style_body,style_richness,style_smoke,style_sweetness,characters,food_paring,bottler,age,country,region,casktype,colouring,vintage,nose,palate,finish,reviews,url
0,Blended Scotch TB-BSW6 Year Old Thompson Bros,Blended Scotch Whisky,70cl / 46%,£34.25,3.0,3.0,0.0,3.0,"Figs, Sultana, Cinnamon, Nutmeg, Toffee, Coffee",NaN,NaN,6 year old,scotland,unknown_scotland,not specified,no,NaN,"Plum sauce, miso, golden sultanas, cinder toff...","Golden sultanas, raisin loaf, buttered Jamaica...","Soft warm spice; allspice, nutmeg and cinnamon...","Stunning stuff for the money, might well be my...",https://www.thewhiskyexchange.com/p/67983/blen...
1,Johnnie Walker Blue Label,Blended Scotch Whisky,70cl / 40%,£158,3.0,3.0,1.0,2.0,"Rose, Hazelnut, Honey, Smoke, Dark Chocolate, ...","Octopus, Monkfish, Fondue",NaN,NaN,scotland,unknown_scotland,not specified,yes,NaN,"Smooth, silky cereal, syrupy flapjacks, polish...",A silky caress. Unexpectedly strong spicy entr...,A perfectly-integrated slow fade of the palate...,Un elixir. || Excellent Whisky. Smooth on the ...,https://www.thewhiskyexchange.com/p/552/johnni...
2,The Big Ben,Blended Scotch Whisky,50cl / 40%,£23.50,NaN,NaN,NaN,NaN,no description,NaN,NaN,NaN,scotland,unknown_scotland,not specified,yes,NaN,"Medium weight, malty nose, with hints of orang...","Medium depth, hints of smokiness and sweetness...",Elegant finish.,Very pleased with the whiskey. Highly recommend,https://www.thewhiskyexchange.com/p/18213/the-...
3,J&B Rare,Blended Scotch Whisky,70cl / 40%,£25.25,NaN,NaN,NaN,NaN,no description,NaN,NaN,NaN,scotland,unknown_scotland,not specified,yes,NaN,no description,no description,no description,Very nice || Was delighted to find this whisky...,https://www.thewhiskyexchange.com/p/544/jb-rare
4,Dimple 15 Year OldLitre,Blended Scotch Whisky,100cl / 43%,£69.95,3.0,3.0,0.0,3.0,"Caramel, Cinnamon, Vanilla, Dark Chocolate",NaN,NaN,15 year old,scotland,unknown_scotland,not specified,NaN,NaN,no description,no description,no description,NaN,https://www.thewhiskyexchange.com/p/79846/dimp...


In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1100 entries, 0 to 1099
Data columns (total 22 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             1100 non-null   object 
 1   main_info        1100 non-null   object 
 2   volume_alcohol   1100 non-null   object 
 3   price            1100 non-null   object 
 4   style_body       618 non-null    float64
 5   style_richness   618 non-null    float64
 6   style_smoke      611 non-null    float64
 7   style_sweetness  618 non-null    float64
 8   characters       1100 non-null   object 
 9   food_paring      30 non-null     object 
 10  bottler          505 non-null    object 
 11  age              532 non-null    object 
 12  country          1100 non-null   object 
 13  region           1100 non-null   object 
 14  casktype         1100 non-null   object 
 15  colouring        469 non-null    object 
 16  vintage          219 non-null    float64
 17  nose          

In [21]:
# 중복 제거
dup_mask = df['name'].duplicated(keep=False)
df_dup = df[dup_mask].copy()
df_unique = df[~dup_mask].copy()

info_cols = [
    'style_body', 'style_richness', 'style_smoke', 'style_sweetness',
    'characters', 'food_paring', 'bottler', 'age',
    'colouring', 'vintage', 'nose', 'palate', 'finish', 'reviews'
]

def select_rows_for_name(g):
    result_list = []

    # 2번 규칙: 같은 name 안에서 (volume_alcohol, price) 조합이 다르면 각각 1개씩은 남긴다
    for (_, _), sub in g.groupby(['volume_alcohol', 'price'], dropna=False):

        if len(sub) == 1:
            result_list.append(sub)
            continue

        # 3번 규칙 위해: info_cols 기준으로 "정보 채워진 개수" 점수 계산
        sub = sub.copy()
        sub['__info_nonnull'] = sub[info_cols].notna().sum(axis=1)

        # 1번 + 3번 규칙:
        # url 제외 + info_cols 제외한 나머지가 완전히 같은 애들끼리 묶어서,
        # 그 안에서 __info_nonnull 이 가장 큰 행 1개만 남긴다.
        base_cols = [
            c for c in sub.columns
            if c not in (['url', '__info_nonnull'] + info_cols)
        ]

        chosen_rows = []
        grouped = sub.groupby(base_cols, dropna=False)

        for _, sub_same in grouped:
            # 정보가 가장 많이 채워진 행 선택 (동점이면 첫 번째)
            best_idx = sub_same['__info_nonnull'].idxmax()
            chosen_rows.append(sub_same.loc[[best_idx]])

        result_list.append(pd.concat(chosen_rows))

    return pd.concat(result_list)

df_dup_clean = (
    df_dup
    .groupby('name', group_keys=False)
    .apply(select_rows_for_name)
)

df = (
    pd.concat([df_unique, df_dup_clean])
    .sort_index()
)
df.drop(columns='__info_nonnull',inplace=True)

# 1️⃣ 제품 기본 정보

### main info

In [22]:
df['main_info'] = df['main_info'].str.lower()

In [23]:
import re

def extract_whisky_type(text):
    if not isinstance(text, str):
        return "unknown"

    t = text.lower()


    patterns = [
        # 1) 가장 구체적인 타입
        ("single pot still", r"\bsingle\s+pot\s+still\b"),

        # 2) 몰트 계열
        ("single malt", r"\bsingle\s+malt\b"),
        ("blended malt", r"\bblended\s+malt\b"),

        ("blended malt", r"\b(pure|vatted)\s+malt\b"),

        # 3) 미국/캐나다 계열
        ("tennessee", r"\btennessee\b"),
        ("bourbon", r"\bbourbon\b"),
        ("canadian", r"\bcanadian\b"),

        # 4) pot still (single pot still보다 후순위)
        ("pot still", r"\bpot\s+still\b"),

        # 5) 기타 곡물 타입
        ("grain", r"\bgrain\b"),
        ("rye", r"\brye\b"),
        ("corn", r"\bcorn\b"),

        # 6) 제일 범용적인 blended (malt 제외)
        ("blended", r"\bblended\b(?!\s+malt)"),
    ]

    for whisky_type, pat in patterns:
        if re.search(pat, t):
            return whisky_type

    return "other"

df["whisky_type"] = df["main_info"].apply(extract_whisky_type)
print(df["whisky_type"].value_counts())

whisky_type
single malt         447
blended             346
blended malt        139
bourbon              43
grain                33
single pot still     32
other                18
rye                  13
tennessee            11
corn                  1
Name: count, dtype: int64


In [24]:
df[df['whisky_type']=='other']

,name,main_info,volume_alcohol,price,style_body,style_richness,style_smoke,style_sweetness,characters,food_paring,bottler,age,country,region,casktype,colouring,vintage,nose,palate,finish,reviews,url,whisky_type
214,Ben Nevis 40 Year OldSingle Blend Bot.2002,single blenddistillery bottling,70cl / 40%,"£1,350",NaN,NaN,NaN,NaN,no description,NaN,distillery bottling,40 year old,scotland,unknown_scotland,not specified,yes,1962.0,no description,no description,no description,NaN,https://www.thewhiskyexchange.com/p/65170/ben-...,other
335,Michter's US*1 Small Batch Bourbon,american whiskey,70cl / 45.7%,£56.75,3.0,3.0,0.0,3.0,"Dried Fruit, Cinnamon, Oak, Vanilla, Toffee",NaN,NaN,NaN,usa,kentucky,not specified,no,NaN,"Appealingly rich notes of barley sugar, raisin...","Rich and spicy, with juicy fruit, cinnamon and...","Welcoming kick of black pepper, then the dried...","Wonderful rich fragrance and flavour, with tho...",https://www.thewhiskyexchange.com/p/26282/mich...,other
345,FEW Bourbon,american whiskey,70cl / 46.5%,£59.95,2.0,2.0,0.0,2.0,"Toffee, Clove, Vanilla, Apple, Ginger",NaN,NaN,NaN,usa,illinois,not specified,no,NaN,no description,no description,no description,The most quintessential example of a highly-ch...,https://www.thewhiskyexchange.com/p/18461/few-...,other
353,Michter's US*1 Single Barrel Straight Rye,american whiskey,70cl / 42.4%,£60.25,3.0,3.0,1.0,2.0,"Black Pepper, Orange, Butterscotch, Oak",NaN,NaN,NaN,usa,kentucky,not specified,no,NaN,"Very aromatic and fresh. Almost minty, with lo...","Incredibly smooth mouthfeel for a rye, this is...",Appealingly dry spiciness – impressive stuff.,Excellent stuff – smooth as silk. || Fantastic...,https://www.thewhiskyexchange.com/p/26283/mich...,other
386,Woodford Reserve Five-Malt Stouted MashMasters...,kentucky malt whiskey,70cl / 45.2%,£119,4.0,4.0,0.0,3.0,"Hazelnut, Milk Chocolate, Cherry, Shortbread, ...",NaN,NaN,NaN,usa,kentucky,not specified,no,NaN,no description,no description,no description,NaN,https://www.thewhiskyexchange.com/p/73201/wood...,other
389,Buffalo Trace White Dog Mash 1Half Bottle,unaged american spirit,37.5cl / 62.5%,£27.50,NaN,NaN,NaN,NaN,"Grass, Oil, Black Pepper, Apple",NaN,NaN,NaN,usa,kentucky,not specified,no,NaN,no description,no description,no description,This exceeded my expectations by far. First of...,https://www.thewhiskyexchange.com/p/12766/buff...,other
769,The House of Suntory Triology Pack3x20cl,suntory,60cl / 42%,£49.25,NaN,NaN,NaN,NaN,no description,NaN,suntory,NaN,japan,unknown_japan,not specified,NaN,NaN,no description,no description,no description,NaN,https://www.thewhiskyexchange.com/p/63405/the-...,other
803,Karuizawa 1999Artist #13 Spirits Shop Selectio...,japanese whisky,70cl / 60%,"£4,000",4.0,4.0,3.0,3.0,"Blood orange, Dark Chocolate, Tobacco, Almond",NaN,NaN,NaN,japan,unknown_japan,sherry butt,NaN,1999.0,no description,no description,no description,NaN,https://www.thewhiskyexchange.com/p/79877/karu...,other
814,Yamazaki 1986Vintage Malt Bot.2004,japanese whisky,70cl / 56%,"£15,000",NaN,NaN,NaN,NaN,no description,NaN,NaN,NaN,japan,unknown_japan,not specified,NaN,1986.0,no description,no description,no description,NaN,https://www.thewhiskyexchange.com/p/69904/yama...,other
833,SMWS 116.19 (Yoichi)1994 20 Year Old,japanese whisky,70cl / 61.3%,"£3,500",NaN,NaN,NaN,NaN,no description,NaN,NaN,20 year old,japan,unknown_japan,puncheon,NaN,1994.0,no description,no description,no description,NaN,https://www.thewhiskyexchange.com/p/63198/smws...,other


In [25]:
df.loc[865,'whisky_type'] =  "blended malt"
df.loc[954,'whisky_type'] =  "blended malt"
df.loc[214,'whisky_type'] =  "blended"
df.loc[345,'whisky_type'] =  "bourbon"
df.loc[335,'whisky_type'] =  "bourbon"
df.loc[353,'whisky_type'] =  "rye"

# 2️⃣ 품질 / 가격 지표

###  volume/ alcohol

In [26]:
df['volume'] = df['volume_alcohol'].str.split(' / ').str[0]
df['alcohol'] = df['volume_alcohol'].str.split(' / ').str[1]

df['volume(cl)'] = df['volume'].str.replace('cl','', regex=False).astype(float)
df['alcohol(%)'] = df['alcohol'].str.replace('%','', regex=False).astype(float)

In [27]:
df['alcohol_density'] = df['alcohol(%)'] * df['volume(cl)']

### price

In [28]:
df['price(£)'] = df['price'].str.replace('£','',regex=False).str.replace(',','',regex=False).astype(float)

In [29]:
df['price_per_cl'] = df['price(£)'] / df['volume(cl)']

### age / vintage / bottler(year)

In [30]:
import re

'''

제품 이름에서 연식 관련 이름 찾기
(age, bottler)

'''


# 0) 전처리: 이름 정규화

def norm_name_lower(s: str) -> str:
    if not isinstance(s, str):
        return ""
    x = s.lower().strip()

    x = re.sub(r"[–—−‐]", "-", x)
    x = re.sub(r"([a-z])(\d{4})\b", r"\1 \2", x)
    x = re.sub(r"(\d{4})\s*[/\-]\s*(\d{4})", r"\1 - \2", x)
    x = re.sub(r"([a-z])\s*(bot\.|btl\.|btld\.|bt\.|b\.)\b", r"\1 \2", x)
    x = re.sub(r"(\d{4})(\d{1,2})\s*(year|yo|yrs?)\b", r"\1 \2 \3", x)
    x = re.sub(r",(?!\s)", ", ", x)
    x = re.sub(r"\s+", " ", x).strip()
    return x



# 1) 패턴 정의

RE_BOT_YEAR    = re.compile(r"\b(bottled|bot\.|btl\.|btld\.|bt\.|b\.)\s*(?:in\s*)?(18\d{2}|19\d{2}|20\d{2})\b")
RE_BOT_DECADE  = re.compile(r"\b(bottled|bot\.|btl\.|btld\.|bt\.|b\.)\s*(18\d0s|19\d0s|20\d0s)\b")
RE_AGE_TEXT    = re.compile(r"\b(\d{1,2})\s*(?:year\s*old|years?|yrs?|yo|y\.?o\.?)\b")


# 2) 파싱 함수 (3개 변수만 추출)

def parse_from_name(name_raw: str):
    out = {
        "bottling_year": None,
        "bottling_decade": None,
        "age_years": None,
    }
    s = norm_name_lower(name_raw)

    # (A) bottling year / decade
    m = RE_BOT_YEAR.search(s)
    if m:
        y = int(m.group(2))
        out["bottling_year"] = y
    else:
        m = RE_BOT_DECADE.search(s)
        if m:
            out["bottling_decade"] = m.group(2)

    # (B) age in years
    m = RE_AGE_TEXT.search(s)
    if m:
        yrs = int(m.group(1))
        out["age_years"] = yrs

    return out



# 3) 3 칼럼 생성

parsed = df["name"].apply(parse_from_name).apply(pd.Series)

for col in ["bottling_year", "bottling_decade", "age_years"]:
    if col not in df.columns:
        df[col] = parsed[col]
    else:
        df[col] = df[col].where(df[col].notna(), parsed[col])

for col in ["bottling_year", "age_years"]:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce").astype("Int16")



# 제목에서 찾은 age 채우기
df['age'] = df['age'].where(df['age'].notna(), df['age_years'])
df['age'] = df['age'].str.extract(r'(\d+)').astype('Int64')
df.drop(columns='age_years', inplace=True)

# bottling_year 26 개 행만 존재 bottling_decade 값으로 변환해서 합침
df['bottling_year'] = (df['bottling_year'] // 10) * 10
df['bottling_decade'] = df['bottling_decade'].fillna(df['bottling_year'])

decade_str =  (
    df['bottling_decade']
    .astype(str)
    .str.strip()
    .str.replace('s', '', regex=False)
)
decade_str = decade_str.replace(['nan', ''], np.nan)
df['bottling_decade'] = pd.to_numeric(decade_str, errors='coerce').astype('Int64')
df.drop(columns='bottling_year',inplace=True)

In [31]:
cols = ["vintage", "bottling_decade", "age"]

combs = df[cols].notna().astype(int).astype(str).agg("".join, axis=1)
print(combs.value_counts().sum())

mapping = {
    "111": "모두 있음",
    "001": "age만 있음",
    "110": "vintage + bottling decade 있음",
    "010": "bottling decade만 있음",
    "000": "아무것도 없음",
    "101": "vintage + age",
    "100": "vintage만 있음",
    "011": "age + bottling decade",
}
combs.map(mapping).value_counts()

1083


,count
아무것도 없음,401
age만 있음,287
vintage + age,166
bottling decade만 있음,111
age + bottling decade,66
vintage만 있음,36
vintage + bottling decade 있음,11
모두 있음,5


In [32]:
 # is_vintage
df["vintage"] = (
    df["vintage"]
    .astype(str)
    .str.extract(r"(\d{4})")
    .astype(float)
    .astype("Int64")
)
df['is_vintage'] = df['vintage'].notna().astype(int)
 # has_age
df['has_age'] = df['age'].notna().astype(int)
 # has_bottling
df['has_bottling_decade'] = df['bottling_decade'].notna().astype(int)
 # is_nas (세가지 모두 NA)
df['is_nas'] = np.where((df['age'].isna()) & (df['vintage'].isna()) & (df['bottling_decade'].isna()), 1, 0)

# 3️⃣ 생산 / 유통 메타 데이터

### region / country

In [33]:
df['region'] = df['region'].str.lower()
df['country'] = df['country'].str.lower()

### bottler

In [34]:
df.bottler.unique()

array([nan, 'compass box', 'distillery bottling', 'berry bros & rudd',
       'watt whisky', 'the artful dodger', 'gleann mor',
       'chivas brothers', 'elixir distillers', 'the last drop',
       'signatory', 'mossburn', 'single cask nation', 'douglas laing',
       'hunter laing', 'duncan taylor', 'new riff distillery',
       'gordon & macphail', 'the whisky exchange', 'ferg and harris',
       'la maison du whisky', 'artful dodger', 'thompson bros',
       'john crabbie', 'wilson & morgan', 'hidden spirits',
       'hart brothers', 'Distillery Bottling', 'speciality drinks ltd',
       'north star spirits', 'finn thomson', 'carn mor', 'jack wiebers',
       'blackadder', 'decadent drinks', "scott's selection",
       'scotch malt whisky society', "cadenhead's", 'mcbain & charles',
       'first cask', 'suntory', 'wemyss', 'the lost distillery company',
       'adelphi', 'whisky works', 'the whisky agency', 'moon import'],
      dtype=object)

In [35]:
import re, unicodedata

# 1. bottler 소문자화, 유니코드 정규화, ' 통일, 다중 공백 제거
def norm_text(s: str) -> str:

    s = "" if pd.isna(s) else str(s)
    s = s.lower()
    s = unicodedata.normalize("NFKD", s)
    s = re.sub(r"[’‘´`']", "'", s)   # 다양한 따옴표를 ' 로 통일
    s = re.sub(r"\s+", " ", s).strip()
    return s
# 이름 정규화 칼럼
df["name_norm"]    = df["name"].map(norm_text)
# 보틀러 정규화 칼럼
df["bottler_norm"] = df["bottler"].map(norm_text)

# ----------------------------------------------------------------

# 2. 동일 보틀러가 여러 표기로 들어온 경우
ALIAS_TO_CANON = {
    "distillery bottling": "distillery bottling",
    "the artful dodger": "artful dodger",
    "artful dodger": "artful dodger",

}

# 동일 보틀러 이름 표기 다른 경우 replace 처리
df["bottler_group"] = df["bottler_norm"].replace(ALIAS_TO_CANON)

#----------------------------------------------------------------
# 3. 이름 속 보틀러 패턴 regex, label dict
IB_PATTERNS = {
    r"\bthompson\s*bros?\b": "thompson bros",
    r"\bcompass\s*box\b": "compass box",
    r"\bwoven\b": "woven",
    r"\bjames\s*eadie\b": "james eadie",
    r"\bberry\s*bros?\.?\s*&\s*rudd\b": "berry bros & rudd",
    r"\bwatt\s*whisky\b": "watt whisky",
    r"\bartful\s*dodger\b": "artful dodger",
    r"\bgleann\s*mor\b": "gleann mor",
    r"\bpure\s*scot\b": "pure scot",
    r"\bcoachbuilt\b": "coachbuilt",
    r"\belixir\s*distillers\b": "elixir distillers",
    r"\bthe\s*last\s*drop\b": "the last drop",
    r"\bwhisky\s*works\b": "whisky works",
    r"\bsignatory\b": "signatory",
    r"\bmossburn\b": "mossburn",
    r"\bsingle\s*cask\s*nation\b": "single cask nation",
    r"\bdouglas\s*laing\b": "douglas laing",
    r"\bhunter\s*laing\b": "hunter laing",
    r"\bduncan\s*taylor\b": "duncan taylor",
    r"\bgordon\s*&\s*macphail\b": "gordon & macphail",
    r"\bthe\s*whisky\s*exchange\b": "the whisky exchange",
    r"\bferg\s*and\s*harris\b": "ferg and harris",
    r"\bla\s*maison\s*du\s*whisky\b": "la maison du whisky",
    r"\bjohn\s*crabbie\b": "john crabbie",
    r"\bwilson\s*&\s*morgan\b": "wilson & morgan",
    r"\bhidden\s*spirits\b": "hidden spirits",
    r"\brest\s*&\s*be\s*thankful\b": "rest & be thankful",
    r"\bhart\s*brothers\b": "hart brothers",
    r"\bspeciality\s*drinks\s*ltd\b": "speciality drinks ltd",
    r"\bnorth\s*star\s*spirits\b": "north star spirits",
    r"\bfinn\s*thomson\b": "finn thomson",
    r"\bcarn\s*mor\b": "carn mor",
    r"\bjack\s*wiebers\b": "jack wiebers",
    r"\bblackadder\b": "blackadder",
    r"\bdecadent\s*drinks\b": "decadent drinks",
    r"\bscott'?s\s*selection\b": "scott's selection",
    r"\bscotch\s*malt\s*whisky\s*society\b|\bsmws\b": "scotch malt whisky society",
    r"\bcadenhead'?s\b": "cadenhead's",
    r"\bmcbain\s*&\s*charles\b": "mcbain & charles",
    r"\bfirst\s*cask\b": "first cask",
    r"\bold\s*perth\b": "old perth",
    r"\bwemyss\b": "wemyss",
    r"\bthe\s*lost\s*distillery\s*company\b": "the lost distillery company",
    r"\bgladstone\s*axe\b": "gladstone axe",
    r"\badelphi\b": "adelphi",
    r"\bangels?'?\s*nectar\b": "angels' nectar",
    r"\bthe\s*whisky\s*agency\b": "the whisky agency",
    r"\bmoon\s*import\b": "moon import",
    r"\bsheep\s*dip\b": "sheep dip",
    r"\btwo\s*stacks\b": "two stacks",
    r"\bjj\s*corry\b|\bj\s*&?\s*j\s*corry\b": "jj corry",
    #OB
    r"\bchivas\s*brothers\b": "chivas brothers",
    r"\bsuntory\b": "suntory",
    r"\bnew\s*riff\s*distillery\b": "new riff distillery",
}
# re.compile
IB_REGEX = [(re.compile(regex), label) for regex, label in IB_PATTERNS.items()]

#4.  보틀러가 null 인 값에 대해서 이름에서 먼저 추론하도록 함
def infer_bottler_from_name(name_norm: str) -> str:
    for rx, lab in IB_REGEX:
        if rx.search(name_norm):
            return lab
    return None

mask_na = df["bottler_group"].isna() | (df["bottler_group"] == "")
df.loc[mask_na, "bottler_group"] = df.loc[mask_na, "name_norm"].map(infer_bottler_from_name)


# 5. 그래도 못 찾은 경우 : OB 기본값
df["bottler_group"] = df["bottler_group"].fillna("distillery bottling")



# ---------------------------------------------------------------------------
OB_WHITELIST = {
    "distillery bottling",
    "chivas brothers",
    "suntory",
    "new riff distillery",
}

# 6.기본은 IB(=1), 화이트리스트면 OB(=0)
df["is_independent"] = (~df["bottler_group"].isin(OB_WHITELIST)).astype("int8")

print(df["bottler_group"].value_counts().head(20))
print(df["is_independent"].value_counts(dropna=False))


bottler_group
distillery bottling           797
signatory                      37
douglas laing                  31
compass box                    27
gordon & macphail              22
elixir distillers              15
berry bros & rudd              13
the whisky exchange            13
watt whisky                     9
woven                           9
old perth                       8
chivas brothers                 7
mossburn                        6
two stacks                      6
thompson bros                   5
single cask nation              5
scotch malt whisky society      5
artful dodger                   5
duncan taylor                   4
decadent drinks                 4
Name: count, dtype: int64
is_independent
0    808
1    275
Name: count, dtype: int64


In [36]:
display(df[df['is_independent']==1]['bottler_group'].value_counts().head(10))
display(df[df['is_independent']==0]['bottler_group'].value_counts())

,count
bottler_group,
signatory,37
douglas laing,31
compass box,27
gordon & macphail,22
elixir distillers,15
berry bros & rudd,13
the whisky exchange,13
watt whisky,9
woven,9


,count
bottler_group,
distillery bottling,797
chivas brothers,7
suntory,3
new riff distillery,1


In [37]:
df.drop(columns='bottler', inplace= True)
df = df.rename(columns={'bottler_norm':'bottler'})

### casktype

In [38]:
import pandas as pd

def simplify_cask(x):
    if x == "not specified":
        return "unknown"
    x = str(x).lower().strip()

    # 1) 특수 목재 / 대체 우드 (
    if any(w in x for w in ["mizunara", "sakura", "cedarwood", "cherry wood",
                            "bog oak", "hickory", "chestnut"]):
        return "alt_wood"

    # 2) sherry 계열
    if any(w in x for w in ["sherry", "oloroso", "pedro ximenez", "px",
                            "amontillado", "manzanilla", "fino", "cream"]):
        return "sherry"

    # 3) bourbon / oak 계열
    if any(w in x for w in ["bourbon", "hogshead", "new oak", "barrel",
                            ]):

        return "bourbon"

    # 4) wine / port 계열
    if any(w in x for w in ["wine", "port", "marsala", "barolo", "madeira",
                            "amarone", "valpolicella", "bolgheri", "bordeaux",
                            "tokaji", "moscatel"]):
        return "wine_port"

    # 5) rum 계열
    if "rum" in x:
        return "rum"

    # 6) brandy / cognac 계열
    if any(w in x for w in ["cognac", "armagnac", "calvados",
                            "brandy", "marnier"]):
        return "brandy_cognac"

    # 7) experimental (beer, liqueur 등)
    if any(w in x for w in ["beer", "liqueur"]):
        return "other_experimental"

    if "oak" in x:
        return "oak_generic"

    # 8) 나머지
    return "others"

df["cask_group"] = df["casktype"].apply(simplify_cask)

In [39]:
df['casktype'].value_counts()

,count
casktype,
not specified,752
sherry,32
hogshead,16
bourbon,13
sherry butt,12
...,...
cedarwood finish,1
blackberry brandy finish,1
apple brandy finish,1


In [40]:
df['cask_group'].value_counts()

,count
cask_group,
unknown,752
sherry,142
bourbon,98
wine_port,41
others,19
rum,12
alt_wood,9
brandy_cognac,8
other_experimental,1


###  colouring

In [41]:
df['colouring'].value_counts()

df['colouring'] = df['colouring'].str.lower()
colouring_map = {'no':False,
                 'yes': True}
df['colouring'] = df['colouring'].map(colouring_map)

df['colouring'] = df['colouring'].fillna('Unknown')

# 4️⃣ 소비 경험 데이터

###  nose/palate/finish/characters

In [42]:
df['is_no_note'] = ((df['characters'] == 'no description') & (df['palate'] == 'no description') & (df['finish'] == 'no description')).astype(int)

### style

In [43]:
# style 지표에서 smoke 에 대한 정보만 없는 제품은 smoke = 0 으로 처리함
smoke_na_mask = (df['style_body'].notna()) & (df['style_richness'].notna()) & (df['style_sweetness'].notna()) & (df['style_smoke'].isna())
df.loc[smoke_na_mask, 'style_smoke'] = 0

# 데이터 저장하기

In [44]:
df.columns

Index(['name', 'main_info', 'volume_alcohol', 'price', 'style_body',
       'style_richness', 'style_smoke', 'style_sweetness', 'characters',
       'food_paring', 'age', 'country', 'region', 'casktype', 'colouring',
       'vintage', 'nose', 'palate', 'finish', 'reviews', 'url', 'whisky_type',
       'volume', 'alcohol', 'volume(cl)', 'alcohol(%)', 'alcohol_density',
       'price(£)', 'price_per_cl', 'bottling_decade', 'is_vintage', 'has_age',
       'has_bottling_decade', 'is_nas', 'name_norm', 'bottler',
       'bottler_group', 'is_independent', 'cask_group', 'is_no_note'],
      dtype='object')

In [45]:
last_cols = ['name','whisky_type','volume(cl)', 'alcohol(%)','alcohol_density', 'price(£)', 'price_per_cl',      # 기본정보
             'style_body','style_richness', 'style_smoke', 'style_sweetness',   # 스타일 지표 4가지
             'country', 'region',                                               # 장소
             'casktype','cask_group',                                           # casktype
             'bottler','bottler_group', 'is_independent',                       # bottler
             'colouring',                                                       # colouring
             'age','vintage','bottling_decade',                                 # 연식
             'is_vintage', 'has_age', 'has_bottling_decade','is_nas',
             'characters','nose', 'palate', 'finish','food_paring','reviews', 'is_no_note','url' # 텍스트 칼럼
]
df[last_cols].head()
df_processed = df[last_cols]
df_processed.to_csv('/content/drive/MyDrive/week_hands_on_project5/whiskey/dataset/whisky_processed.csv', index=False)